<img src="Images/Logo.png" alt="Logo NSI" style="float:right">

<h1 style="text-align:center">Chapitre 19 : Requêtes SQL et mises à jour</h1>

Nous avons vu comment créer des tables et les remplir.  
Nous avons maintenant une base de données, c'est-à-dire un ensemble de tables, contenant des données **cohérentes** vis à vis de nos contraintes d'intégrité.  

Nous allons maintenant voir deux autres utilisations d'un SGBD :
* la sélection de données
* la mise à jour des données

La sélection va consister en l'écriture de **requêtes SQL** permettant de trouver toutes les données de la base vérifiant un certain critère.  
Le premier rôle du programmeur de bases de données va donc être celui qui consiste à traduire des questions que l'on se pose sur les données du langage naturel au langage SQL, afin que le SGBD puisse y répondre.  

Reprenons le fil conducteur de notre introduction aux bases de données, à savoir celui de la médiathèque municipale.  
Quelles questions peut-on poser à la base de données?  
Il semble naturel que le SGBD puisse répondre aux questions suivantes, nécessaire au bon fonctionnement
de la médiathèque.
* Étant donné un code barre, quels sont les livres empruntés par l'utilisateur correspondant?
* Étant donné un ISBN, le livre correspondant est-il emprunté?
* Quels sont les utilisateurs en retard, c'est-à-dire ceux dont la date de retour est inférieure à une date donnée?
* Quels sont tous les livres écrits par Voltaire qui ne sont pas empruntés?
* Quel est le nombre total de livres empruntés?

Une autre fonction importante du SGBD est la mise à jour des données.  
Elle peut consister en une modification d'une ligne existante (par exemple, pour changer l'adresse d'un utilisateur ayant déménagé, sans modifier son code barre, son nom ou son e-mail) ou une suppression (par exemple lorsqu'un utilisateur rend un livre, il faut supprimer la ligne correspondante dans la table `emprunt`).

## Sélection de données

### Requête sur une table
Commençons par une requête simple. On considère la table [`livre`](Fichiers/livre.csv), créée par l'ordre suivant:

```sql
CREATE TABLE livre (titre VARCHAR(300) NOT NULL, editeur VARCHAR(90) NOT NULL, 
                     annee INT NOT NULL, isbn CHAR(14) PRIMARY KEY);
```

On souhaite trouver [les titres de tous les livres publiés après 1990 dans la base de données de la médiathèque](Fichiers/resultat_1.csv).  
Une telle requête peut s'écrire en SQL :

```sql
SELECT titre FROM livre WHERE annee >= 1990;
```

|               titre               |
|:---------------------------------:|
| Les Aventures de Huckleberry Finn |
|        Fondation et Empire        |
|               Akira               |
|             Les Robots            |
|      Astérix chez les Pictes      |
|        Les Monades urbaines       |
| ...                               |

```

Result: 112 enregistrements ramenés en 9ms
```

Dans cette requête, la partie `FROM livre` indique la table sur laquelle porte la requête.  
La partie `WHERE ...` indique que l'on ne sélectionne que les lignes de la table `livre` pour lesquelles la valeur de l'attribut `annee` est plus grande que 1990.  
Enfin, la partie `SELECT titre` indique qu'on ne veut renvoyer que les valeurs de l'attribut `titre` des lignes trouvées.  
On remarque que le résultat d'une requête `SELECT ...` est une table, ici possédant une unique colonne `titre`.

### Clause WHERE
L'expression se trouvant dans la partie `WHERE` doit être une expression booléenne.  
Elle peut être construite à partir d'opérateurs de comparaison (`<`, `<=`, `>`, `>=`, `=` et `<>`), d'opérateurs arithmétiques (`+`, `-`, `*`, `/`, `%`), de constantes, de noms d'attributs, d'opérateurs logiques (`AND`, `OR` et `NOT`) et d'opérateurs spéciaux tels que l'opérateur de comparaison de textes `LIKE`.
Par exemple, si l'on souhaite afficher [les titres de tous les livres publiés par Dargaud entre 1970 et 1980](Fichiers/resultat_2.csv), on pourra écrire:

```sql
SELECT titre FROM livre WHERE annee >= 1970 AND annee <= 1980 AND editeur = 'Dargaud';
```

|               titre               |
|:---------------------------------:|
| Astérix chez les Belges |

```

Result: 1 enregistrements ramenés en 3ms
```

Si l'utilisation de l'égalité `=` est appropriée ici, on pourrait vouloir faire une requête approchée.  
Par exemple, trouver [les titres des livres qui contiennent le mot `'Astérix'` dans le titre](Fichiers/resultat_3.csv).  
Une telle requête s'écrira :

```sql
SELECT titre FROM livre WHERE titre LIKE '%Astérix%';
```

|            titre            |
|:---------------------------:|
|   Astérix chez les Pictes   |
|     Astérix et Cléopâtre    |
|  Le Tour de Gaule d'Astérix |
|   Astérix et les Normands   |
|       Astérix en Corse      |
|     Astérix légionnaire     |
| Astérix et la Transitalique |
|     L'Odyssée d'Astérix     |
|   Astérix chez les Bretons  |
|   Astérix chez les Belges   |

```
Result: 10 enregistrements ramenés en 4ms
```



### Clause SELECT
La clause `SELECT` peut prendre diverses formes.
* La première est celle où l'on liste explicitement les attributs que l'on désire renvoyer.  
Si on veut renvoyer [les titres et l'ISBN des livres publiés après 1990](Fichiers/resultat_4.csv), on écrira ceci:

```sql
SELECT titre, isbn FROM livre WHERE annee >= 1990;
```

|               titre               |      isbn      |
|:---------------------------------:|:--------------:|
| Les Aventures de Huckleberry Finn | 978-2081509511 |
| Fondation et Empire               | 978-2207249123 |
| Akira                             | 978-2723428262 |
| Les Robots                        | 978-2745989857 |
| Astérix chez les Pictes           | 978-2864972662 |
| Les Monades urbaines              | 978-2221197691 |
| Les Voyages de Gulliver           | 978-2335008586 |
| Lolita                            | 978-0141391601 |
| La Nuit des temps                 | 978-2258116429 |
| Ravage                            | 978-2072534911 |
| Les Lauriers de César             | 978-2012101500 |
|                ...                |                |

```
Result: 112 enregistrements ramenés en 17ms
```

Le résultat est de nouveau une table, mais cette fois avec les colonnes (ou attributs) `titre` et `isbn`.  
Il est possible de renommer les colonnes au moyen du mot clé `AS`.

```sql
SELECT titre AS le_titre, isbn AS num_serie FROM livre WHERE annee >= 1990;
```


|               le_titre               |      num_serie      |
|:---------------------------------:|:--------------:|
| Les Aventures de Huckleberry Finn | 978-2081509511 |
| Fondation et Empire               | 978-2207249123 |
| Akira                             | 978-2723428262 |
| Les Robots                        | 978-2745989857 |
| Astérix chez les Pictes           | 978-2864972662 |
| Les Monades urbaines              | 978-2221197691 |
| Les Voyages de Gulliver           | 978-2335008586 |
| Lolita                            | 978-0141391601 |
| La Nuit des temps                 | 978-2258116429 |
| Ravage                            | 978-2072534911 |
| Les Lauriers de César             | 978-2012101500 |
|                ...                |                |

```
Result: 112 enregistrements ramenés en 13ms
```

Ainsi, si la clause `WHERE` d'une requête permet de restreindre les lignes de la table que l'on renvoie (en ne gardant que celle vérifiant la condition), la clause `SELECT` permet de restreindre la liste des colonnes.

* La seconde forme de la clause `SELECT` est celle que l'on utilise lorsqu'on veut conserver [toutes les colonnes](Fichiers/resultat_5.csv). En effet, il serait fastidieux de récrire toutes les colonnes d'une table.  
On peut utiliser à cette fin le symbole `*` qui signifie `toutes les colonnes de la table`.

```sql
SELECT * FROM livre WHERE annee >= 1990;

```
| titre                             | editeur              | annee | isbn           |
|-----------------------------------|----------------------|-------|----------------|
| Les Aventures de Huckleberry Finn | Flammarion           | 2020  | 978-2081509511 |
| Fondation et Empire               | Editions Denoël      | 1999  | 978-2207249123 |
| Akira                             | Glénat               | 2000  | 978-2723428262 |
| Les Robots                        | Editions Milan       | 2017  | 978-2745989857 |
| Astérix chez les Pictes           | Editions Albert René | 2013  | 978-2864972662 |
| Les Monades urbaines              | Robert Laffont       | 2016  | 978-2221197691 |
| Les Voyages de Gulliver           | Primento             | 2015  | 978-2335008586 |
| Lolita                            | Penguin UK           | 2012  | 978-0141391601 |
| La Nuit des temps                 | Presses de la Cité   | 2014  | 978-2258116429 |
| Ravage                            | Editions Gallimard   | 2014  | 978-2072534911 |
| Les Lauriers de César             | Educa Books          | 2008  | 978-2012101500 |
| ...                               |                      |       |                |

```
Result: 112 enregistrements ramenés en 18ms
```

* La troisième utilisation de la clause `SELECT` est celle permettant d'appeler des **fonctions d'agrégation**.  
Ces dernières permettent d'appliquer une fonction à l'ensemble des valeurs d'une colonne et de renvoyer le résultat comme une table ayant une seule case (une ligne et une colonne).   
Voici quelques unes de ces fonctions: 
    * `COUNT` qui permet d'obtenir le nombre de résultats
    * `AVG` (pour l'anglais average) qui permet de calculer la moyenne d'une colonne
    * `SUM` qui permet d'en faire la somme
    * `MIN` et `MAX` qui permettent de trouver respectivement le minimum et maximum d'une colonne. 
    
Si on souhaite savoir [combien de livres contiennent la chaîne `'Astérix'` dans leur titre](Fichiers/resultat_6.csv) (plutôt que de renvoyer ces titres), on écrira la requête suivante:

```sql
SELECT COUNT(titre) AS total FROM livre WHERE titre LIKE '%Astérix%';
```

| total                             | 
|-----------------------------------|
| 10 |

```
Result: 1 enregistrements ramenés en 2ms
```

Notons que nous avons choisi de renommer la colonne.  
En effet, le résultat n'étant pas directement une colonne d'une table existante, les SGBD choisissent un nom arbitraire, souvent peu parlant.  
La fonction `COUNT` ne faisant que compter la taille de la colonne, elle peut s'appliquer à n'importe quel nom de colonne et même au symbole `*`.  
Il est donc courant d'écrire une requête de la forme

```sql
SELECT COUNT(*) AS total FROM livre WHERE titre LIKE '%Astérix%';
```
qui donnera le même résultat que précédemment.  
Les fonctions `AVG` et `SUM` ne peuvent s'appliquer qu'à des colonnes dont le domaine est un nombre.  
On peut écrire par exemple

```sql
SELECT SUM(annee) AS somme FROM livre;
```

| somme                             | 
|-----------------------------------|
| 256701 |

```
Result: 1 enregistrements ramenés en 6ms
```

```sql
SELECT AVG(annee) AS moyenne FROM livre;
```

| moyenne                             | 
|-----------------------------------|
| 2005.4765625 |

```
Result: 1 enregistrements ramenés en 7ms
```

Ici, en l'absence de clause `WHERE`, toutes les lignes sont sélectionnées.  
La somme et la moyenne des années sont calculées et renvoyées comme une table.  

Enfin, les fonctions `MIN()` et `MAX()` peuvent s'appliquer sur n'importe quelle colonne et la comparaison pour son type sera utilisée pour déterminer le plus petit ou le plus grand élément.

```sql
SELECT MIN(annee) AS inf FROM livre;
```

| inf                             | 
|-----------------------------------|
| 1933 |

```
Result: 1 enregistrements ramenés en 2ms
```

```sql
SELECT MAX(annee) AS sup FROM livre;
```

| sup                             | 
|-----------------------------------|
| 2020 |

```
Result: 1 enregistrements ramenés en 2ms
```




## Sources :
* Balabonski Thibaut, et al. 2020. *Spécialité Numérique et sciences informatiques : 24 leçons avec exercices corrigés - Terminale - Nouveaux programmes*. Paris. Ellipse
* [SQLite](https://www.sqlite.org/index.html)
* [DB Browser for SQLite](https://sqlitebrowser.org/)
* [Base de données de la médiatèque](Fichiers/mediatheque.sql)